# Lab 3 - Nim  
## Task 3.2 - An agent using evolved rules

Here for semplicity, I will consider the parameter k always equal to None

In [9]:
import logging
import random
from copy import deepcopy
from nim import Nimply, Nim
from play_nim import nim_sum, random_action, evaluate

In [10]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

## Implementation

### Individual

In [86]:
class EvolvedPlayer():
  """
    This played uses GA to evolve some rules
    that lets him play the game (hopefully better every time).  

    The genome will be a list of rules that will be applyed 
    in order. The information lies inside the order of the rules, that 
    can change with genetic operations (XOVER and MUT).
  """

  def __init__(self, nim: Nim, genome=None):
    self._k = nim._k
    self.__collect_info(nim) # Cooked info
    self.rules = self.__rules()
    if genome:
      assert len(genome) == len(self.rules)
      self.genome = genome
    else: 
      self.genome = list(self.rules.keys())
      random.shuffle(self.genome) 


  def __collect_info(self, nim: Nim):
    self.n_rows_left = len([r for r in nim._rows if r > 0])
    self.sorted_rows = sorted([(r, n) for r, n in enumerate(nim._rows) if n > 0], key=lambda r: -r[1])
    self.avg_obj_per_row = sum(nim._rows) / len(nim._rows)    


  def __rules(self):
    """
    Returns a set of fixed rules as a dicitonary.  
    The dictionary will be as follows:
      - key: id as incremental number
      - value: tuple with (condition, action), where 
        - condition = boolean condition that has to be true in order to perform the action
        - action = Nimply action 
    """

    assert self.n_rows_left
    assert self.sorted_rows
    assert self.avg_obj_per_row
    
    ### Conditions and Actions ###

    # 1 row left --> take the entire row
    def c1(self): 
      return self.n_rows_left == 1
    def a1(self):
      return Nimply(self.sorted_rows[0][0], self.sorted_rows[0][1])

    # 2 rows left --> leave the same number of objs
    def c2(self):
      return self.n_rows_left == 2
    def a2(self):
      Nimply(self.sorted_rows[0][0], self.sorted_rows[0][1] - self.sorted_rows[1][1])

    # 2 rows left and len longest row > avg --> leave one obj in the higher row
    def c3(self):
      return self.n_rows_left == 2 and self.sorted_rows[0][1] > self.avg_obj_per_row
    def a3(self):
      return Nimply(self.sorted_rows[0][1], self.sorted_rows[0][1] - 1)

    # 3 rows left --> take the entire max row
    def c4(self):
      return self.n_rows_left == 3
    def a4(self):
      return Nimply(self.sorted_rows[0][0], self.sorted_rows[0][1])

    # 3 rows left --> take leave the longest row with one obj
    def c5(self):
      return self.n_rows_left == 3 and self.sorted_rows[0][1] > 1
    def a5(self):
      return Nimply(self.sorted_rows[0][0], self.sorted_rows[0][1] - 1)
    
    # avg < max+1 --> take the longest row
    def c6(self):
      return self.avg_obj_per_row < self.sorted_rows[0][1] + 1
    def a6(self):
      return Nimply(self.sorted_rows[0][0], self.sorted_rows[0][1])

    # default -> take one obj from the longest row
    def c7(self):
      return True
    def a7(self):
      return Nimply(self.sorted_rows[0][0], 1)


    ### Rule dictionary ###
    dict_rules = {
      1: (c1, a1),
      2: (c2, a2),
      3: (c3, a3),
      4: (c4, a4),
      5: (c5, a5),
      6: (c6, a6),
      7: (c7, a7)
    }
  
    return dict_rules


  def ply(self, nim: Nim):
    """
      Check the rules in order. The first rule that matches will be applyed
    """
    # Update the informations
    self.__collect_info(nim)

    # Apply a rule
    for key in self.genome:
      cond, act = self.rules[key]
      if cond(self):
        return act(self)

  
  def cross_over(self, partner, nim):
    """ 
      Cycle crossover: choose two loci l1 and l2 (not included) and copy the segment
      between them from p1 to p2, then copy the remaining unused values
    """
    locus1 = random.randint(0, len(self.genome)-1)
    while (locus2 := random.randint(0, len(self.genome)-1)) == locus1:
      pass
    if locus1 > locus2:
      tmp = locus1
      locus1 = locus2
      locus2 = tmp

    # Segment extraction
    segment_partner = partner.genome[locus1:locus2] # slice
    alleles_left = [a for a in self.genome if a not in segment_partner]
    #random.shuffle(alleles_left)

    # Create the offspring genome
    piece1 = alleles_left[:locus1]
    piece2 = alleles_left[locus1:]
    offspring_genome = piece1 + segment_partner + piece2
    
    return EvolvedPlayer(nim, offspring_genome)


  def mutation(self):
    """
      Swap mutation: alleles in two random loci are swapped
    """
    locus1 = random.randint(0, len(self.genome)-1)
    while (locus2 := random.randint(0, len(self.genome)-1)) == locus1:
      pass
    
    # Swap mutation
    tmp = self.genome[locus1]
    self.genome[locus1] = self.genome[locus2]
    self.genome[locus2] = tmp


### Evolution

In [ ]:
def initial_population(population_size: int, nim: Nim):
  population = []
  for i in range(population_size):
    population.append(EvolvedPlayer(nim))
  
  return population

In [ ]:
def genetic_algorithm():
  pass

## Play

In [ ]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 50
